## adv_statistics.py

In [ ]:
#still working...
#different statistics over low level features

def negentropy(A):
    ne = []
    for u in A:
        a1 = 2
        G1 = 1/a1*np.log(np.cosh(2*u))
        v = np.random.normal(0, 1, len(u))
        G2 = -np.exp(-np.power(v,2)/2)
        ne.append(np.power(np.mean(G1)-np.mean(G2),2))
    return np.array(ne)

#good for discrete features
def differences_entropy(A):
    dH = []
    for f in A:
        Y = np.diff(f)
        P = []
        for yk in Y:
            ntimes = Y.tolist().count(yk)
            P.append(ntimes/len(Y))
        print P
        dH.append(entropy(Y)/np.log(len(Y)))
    return np.array(dH)

## dataset.py

In [ ]:
from pyAudioAnalysis import audioBasicIO#用于读取音频文件
import os
import subprocess as sp
import itertools

class Dataset:

    # Dataset object is composed of:
    # data 
    # targets
    # train and test sets for cross validation
    # classes dictionary to map classes to numbers

    def __init__(self,path,decode):
        self.classes = {0:'W', 1:'L', 2:'E', 3:'A', 4:'F', 5:'T', 6:'N'}#一共有7种情绪，用0~6编号替代
        self.get_berlin_dataset(path)

    def get_berlin_dataset(self,path):
        males = ['03','10','11','12','15']
        females = ['08','09','13','14','16']
        classes = {v: k for k, v in self.classes.iteritems()}
        self.targets = []; self.data = []; self.train_sets = []; self.test_sets = []; get_data = True
        for speak_test in itertools.product(males,females):#test_couples:
            i = 0; train = []; test = [];
            for audio in os.listdir(path):
                audio_path = os.path.join(path,audio)
                [Fs,x] = audioBasicIO.readAudioFile(audio_path)
                if get_data:
                    self.data.append((x,Fs))
                    self.targets.append(classes[audio[5]])
                if audio[:2] in speak_test:
                    test.append(i)
                else:
                    train.append(i)
                i = i + 1
            self.train_sets.append(train)
            self.test_sets.append(test)
            get_data = False




## emorecgnition.py

In [ ]:
from pyAudioAnalysis import audioFeatureExtraction
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from sklearn import svm
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from argparse import ArgumentParser
import numpy as np
from scipy import spatial
from scipy.stats import entropy
import scipy.io as sio
import numpy as np
import pickle
from time import sleep
import sys
from dataset import Dataset
from preprocessing import Eigenspectrum, Preprocessor
#from adv_statistics import negentropy, differences_entropy

if __name__ == '__main__':
    #warnings.filterwarnings('ignore')
    #这里命令行参数解析貌似有点问题
    parser = ArgumentParser()
    parser.add_argument("-d", "--dataset", dest="db_type", default="berlin")
    parser.add_argument("-p", "--dataset_path", dest="path", default="")
    parser.add_argument("-l", "--load_data", action="store_true", dest="load_data")
    parser.add_argument("-e", "--extract_features", action="store_true", dest="extract_features")
    parser.add_argument("-s", "--speaker_indipendence", action="store_true", dest="speaker_indipendence")
    parser.add_argument("-i", "--plot_eigenspectrum", action="store_true", dest="plot_eigenspectrum")
    (options, args) = parser.parse_args(sys.argv)
    load_data = options.load_data
    extract_features = options.extract_features
    db_type = options.db_type
    speaker_indipendence = options.speaker_indipendence
    path = options.path
    plot_eigenspectrum = options.plot_eigenspectrum

    if load_data:
        print("Loading data from " + db_type + " dataset...")
        if db_type != 'berlin':
            sys.exit("Dataset not registered. Please create a method to read it")

        db = Dataset(path,db_type,decode=False)

        print("Saving " + db_type + " dataset info to file...")
        pickle.dump(db, open(db_type + '_db.p', 'wb')) 
    else:
        print("Getting data from " + db_type + " dataset...")
        db = pickle.load(open(db_type + '_db.p', 'rb'))

    n_samples = len(db.targets)
    print("Number of dataset samples: " + str(n_samples))

    if extract_features:
        win_size = 0.04
        step = 0.01
        Fglobal = []
        i = 0
        for (x,Fs) in db.data:
            F = audioFeatureExtraction.stFeatureExtraction(x, Fs, win_size*Fs, step*Fs)
            Fglobal.append( np.concatenate((np.mean(F, axis=1),
                                                np.std(F, axis=1))))

            sys.stdout.write("\033[F") # cursor up one line
            i = i+1; print("Extracting features " + str(i) + '/' + str(n_samples) + " from data...")

        print("Saving features to file...")
        pickle.dump(Fglobal, open(db_type + '_features.p', 'wb')) 
    else:
        print("Getting features from files...")
        Fglobal = pickle.load(open(db_type + '_features.p', 'rb'))

    Fglobal = np.array(Fglobal)
    y = np.array(db.targets)

    # evaluating SVM using cross validation
    print("Evaluating model with cross validation...")

    if speaker_indipendence:
        k_folds = len(db.test_sets)
        splits = zip(db.train_sets,db.test_sets)
    else:
        k_folds = 10
        sss = StratifiedShuffleSplit(n_splits=k_folds, test_size=0.2, random_state=1)
        splits = sss.split(Fglobal, y)

    # setting preprocessing
    pp = Preprocessor('standard',n_components=50)
    n_classes = len(db.classes)
    clf = OneVsRestClassifier(svm.SVC(kernel='rbf',C=10, gamma=0.01))
    prfs = []; scores = []; acc = np.zeros(n_classes)
    mi_threshold = 0.0
    for (train,test) in splits:
        # selecting features using mutual information
        Ftrain = Fglobal[train]; Ftest = Fglobal[test]
        f_subset = pp.mutual_info_select(Ftrain,y[train],mi_threshold)
        Ftrain = Ftrain[:,f_subset]; Ftest = Ftest[:,f_subset]

        #standard transformation
        (Ftrain,Ftest) = pp.standardize(Ftrain,Ftest)

        # eigenspectrum over all data
        if plot_eigenspectrum:
            es = Eigenspectrum(Ftrain)
            es.show()

        (Ftrain,Ftest) = pp.project_on_pc(Ftrain,Ftest)

        clf.fit(Ftrain, y[train])
        ypred = clf.predict(Ftest)

        #print clf.score(Ftest, y[test])
        scores.append(clf.score(Ftest, y[test]))
        #print precision_recall_fscore_support(y[test], ypred)
        prfs.append(precision_recall_fscore_support(y[test], ypred))

    # mean total accuracy
    print("\nAccuracy =  %0.2f (%0.2f)\n" % (np.mean(scores), np.std(scores)))

    #mean per class precision and recall 
    mean_prec = np.zeros((1,n_classes))
    mean_recall = np.zeros((1,n_classes))
    precs = []; recalls = []
    for mat in prfs:
        precs.append(mat[0])
        recalls.append(mat[1])
        mean_prec = mean_prec + mat[0]
        mean_recall = mean_recall + mat[1]
    mean_prec = mean_prec[0] / k_folds
    mean_recall = mean_recall[0] / k_folds

    #mean total recall and precision
    precs = np.array(precs)
    recalls = np.array(recalls)
    prec_mean = np.mean(precs,axis=0)
    prec_std = np.std(precs,axis=0)
    recall_mean = np.mean(recalls,axis=0)
    recall_std = np.std(recalls,axis=0)
    print("Recall %0.2f (%0.2f)" % (np.mean(recall_mean), np.std(recall_mean)))
    print("Precision: %0.2f (%0.2f)\n" % (np.mean(prec_mean), np.std(prec_mean)))

    for i in range(0,n_classes):
        print(db.classes[i] + " precision = %0.2f (%0.2f)" % (prec_mean[i],prec_std[i]))
        print(db.classes[i] + " recall = %0.2f (%0.2f)\n" % (recall_mean[i],recall_std[i]))


## preprocessing.py

In [ ]:
from scipy.linalg import eigh
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif

class Eigenspectrum:

    # Eigenspectrum is composed of:
    # eigenvalues
    # % info for each eigenvalue
    # you can plot it using show()

    def __init__(self, F):
        self.eigenvalues = eigh(np.cov(F.T),eigvals_only=True)
        total_inf = np.sum(self.eigenvalues)
        self.info = self.eigenvalues/total_inf*100

    def show(self):
        info = self.info[::-1]
        plt.plot(range(0,len(self.eigenvalues)),info)
        plt.title('Eigenspectrum')
        plt.show()

class Preprocessor:

    # Preprocessor is composed of:
    # scaler (minmax or standard)
    # a pc projector
    # you can also remove features using mutual_info_select()

    def __init__(self,scaler_type,n_components):
        if scaler_type == "standard":
            self.scaler = StandardScaler()
        elif scaler_type == "minmax":
            self.scaler = MinMaxScaler()
        self.pca = PCA(n_components)

    def standardize(self,Ftrain,Ftest):
        Ftrain_std = self.scaler.fit_transform(Ftrain)
        Ftest_std = self.scaler.transform(Ftest)
        return (Ftrain_std,Ftest_std)

    def project_on_pc(self,Ftrain,Ftest):
        Ftrain_pca = self.pca.fit_transform(Ftrain)
        Ftest_pca = self.pca.transform(Ftest)
        return (Ftrain_pca,Ftest_pca)

    def mutual_info_select(self,F,y,threshold):
        mi = list(enumerate(mutual_info_classif(F,y)))
        f_best = []
        for (ind,rank) in mi:
            if rank > threshold:
                f_best.append(ind)
        return f_best





## 简版
先写一个脚本，将wav中不同类别的语音分开保存

Emo-DB数据集数据命名方式  
03a01Fa.wav  
只需要关注倒数第二个字符，它代表着情绪  
W表示生气、L表示无聊、E表示恶心、A表示愤怒、F表示高兴、T表示伤心、N表示中性

In [28]:
def filing(path,root):
    import os,shutil
    file= os.listdir(path)
    dic= dict({'W':'angry','L':'boring','E':'disgust','A':'axiety','F':'happy','T':'sad','N':'neutral'})
    for item in file:
        A,B= item.split('.')
        key= A[-2] 
        temp= os.path.join(root, dic[key])
        if not os.path.exists(temp):
            os.mkdir(temp)
        else:
            shutil.copy(os.path.join(path, item),os.path.join(temp,item))

In [29]:
file_path= r'C:\Users\Administrator\Desktop\EMO-DB\wav'
root_path= r'C:\Users\Administrator\Desktop\wav'
filing(file_path, root_path)

In [30]:
from pyAudioAnalysis import audioFeatureExtraction
from pyAudioAnalysis import audioBasicIO
import numpy as np
def train(wave,win_size, step_size):
    file= os.listdir(wave)
    data= []
    label= []
    for i in file:
        each_data= []
        temp= os.path.join(wave,i)
        for j in os.listdir(temp):
            [Fs, x]= audioBasicIO.readAudioFile(os.path.join(temp,j))
            feature, feature_name=audioFeatureExtraction.stFeatureExtraction(x, Fs, win_size*Fs, step_size*Fs)
            each_data.append(np.concatenate(np.mean(feature,axis=1),np.std(feature, axis=1)))
        data.append(each_data)
        label.append(i)

In [32]:
[Fs, x]=audioBasicIO.readAudioFile(r'C:\Users\Administrator\Desktop\wav\angry\03a02Wb.wav')
feature, feature_name= audioFeatureExtraction.stFeatureExtraction(x, Fs, 0.025*Fs, 0.01*Fs)

In [41]:
np.concatenate((np.mean(feature,axis=1),np.std(feature, axis=1)),axis=0)

array([ 1.76643991e-01,  1.09479445e-02,  2.94315028e+00,  2.92977119e-01,
        2.32565686e-01,  1.28156261e+00,  1.19125002e-02,  3.25690476e-01,
       -2.92608934e+01,  1.26744341e+00,  3.66709971e-01,  5.69478920e-03,
       -1.75188224e-01, -2.03790474e-01,  1.94795207e-02,  4.47890953e-03,
       -2.37139206e-01,  5.30750588e-02, -1.39822202e-01,  2.99010261e-02,
       -1.07777340e-01,  1.08306065e-02,  2.59741814e-03,  2.88713433e-02,
        4.22013603e-03,  1.45342057e-02,  6.24775808e-03,  6.44403007e-02,
        3.42337667e-03,  4.85564685e-03,  9.51100981e-03,  3.65615371e-02,
        2.66389216e-03,  3.16156411e-02,  1.67446953e-01,  1.85002298e-02,
        4.14269131e-01,  1.34732579e-01,  3.44960686e-02,  7.83674009e-01,
        2.45284685e-02,  2.64638791e-01,  4.66427560e+00,  1.20528791e+00,
        1.15387722e+00,  8.62239317e-01,  4.37379073e-01,  4.30020799e-01,
        3.74374244e-01,  3.48501112e-01,  3.91568739e-01,  3.62246546e-01,
        3.02480243e-01,  

In [38]:
np.mean(feature, axis=1)

array([ 1.76643991e-01,  1.09479445e-02,  2.94315028e+00,  2.92977119e-01,
        2.32565686e-01,  1.28156261e+00,  1.19125002e-02,  3.25690476e-01,
       -2.92608934e+01,  1.26744341e+00,  3.66709971e-01,  5.69478920e-03,
       -1.75188224e-01, -2.03790474e-01,  1.94795207e-02,  4.47890953e-03,
       -2.37139206e-01,  5.30750588e-02, -1.39822202e-01,  2.99010261e-02,
       -1.07777340e-01,  1.08306065e-02,  2.59741814e-03,  2.88713433e-02,
        4.22013603e-03,  1.45342057e-02,  6.24775808e-03,  6.44403007e-02,
        3.42337667e-03,  4.85564685e-03,  9.51100981e-03,  3.65615371e-02,
        2.66389216e-03,  3.16156411e-02])

In [39]:
np.std(feature,axis=1)

array([0.16744695, 0.01850023, 0.41426913, 0.13473258, 0.03449607,
       0.78367401, 0.02452847, 0.26463879, 4.6642756 , 1.20528791,
       1.15387722, 0.86223932, 0.43737907, 0.4300208 , 0.37437424,
       0.34850111, 0.39156874, 0.36224655, 0.30248024, 0.35548318,
       0.32233058, 0.0249163 , 0.00620586, 0.03810156, 0.00654186,
       0.02442344, 0.01205343, 0.10919839, 0.00629853, 0.00752354,
       0.01395993, 0.05403936, 0.00506946, 0.02765093])

In [54]:
from pyAudioAnalysis import audioFeatureExtraction
from pyAudioAnalysis import audioBasicIO
import numpy as np
import os
def train(wave,win_size, step_size):
    file= os.listdir(wave)
    data= []
    label= []
    for i in file:
        each_data= []
        temp= os.path.join(wave,i)
        for j in os.listdir(temp):
            [Fs, x]= audioBasicIO.readAudioFile(os.path.join(temp,j))
            feature,feature_name=audioFeatureExtraction.stFeatureExtraction(x, Fs, win_size*Fs, step_size*Fs)
            each_data.append(np.concatenate((np.mean(feature,axis=1),np.std(feature, axis=1)),axis=0))
            label.append(i)
        data.append(each_data)
    return data, label
wave= r'C:\Users\Administrator\Desktop\wav'
data, label=train(wave, 0.025, 0.01)

In [36]:
np.save(r'C:\Users\Administrator\Desktop\test.txt',data)

In [38]:
data[0][0]

array([ 1.76643991e-01,  1.09479445e-02,  2.94315028e+00,  2.92977119e-01,
        2.32565686e-01,  1.28156261e+00,  1.19125002e-02,  3.25690476e-01,
       -2.92608934e+01,  1.26744341e+00,  3.66709971e-01,  5.69478920e-03,
       -1.75188224e-01, -2.03790474e-01,  1.94795207e-02,  4.47890953e-03,
       -2.37139206e-01,  5.30750588e-02, -1.39822202e-01,  2.99010261e-02,
       -1.07777340e-01,  1.08306065e-02,  2.59741814e-03,  2.88713433e-02,
        4.22013603e-03,  1.45342057e-02,  6.24775808e-03,  6.44403007e-02,
        3.42337667e-03,  4.85564685e-03,  9.51100981e-03,  3.65615371e-02,
        2.66389216e-03,  3.16156411e-02,  1.67446953e-01,  1.85002298e-02,
        4.14269131e-01,  1.34732579e-01,  3.44960686e-02,  7.83674009e-01,
        2.45284685e-02,  2.64638791e-01,  4.66427560e+00,  1.20528791e+00,
        1.15387722e+00,  8.62239317e-01,  4.37379073e-01,  4.30020799e-01,
        3.74374244e-01,  3.48501112e-01,  3.91568739e-01,  3.62246546e-01,
        3.02480243e-01,  

In [46]:
np.savetxt(r'C:\Users\Administrator\Desktop\test.txt',data[0])

In [50]:
f= open(r'C:\Users\Administrator\Desktop\test.txt','w+')
for i1 in range(len(data)):
    arrs= []
    temp= data[i1]
    for i2 in range(len(temp)):
        joint= temp[i2]
        arrs.append(joint)
        for i3 in range(len(temp[0])):
            strNum= str(joint[i3])
            f.write(strNum)
            f.write(' ')
        f.write('\n')
f.close()

In [51]:
Data= np.loadtxt(r'C:\Users\Administrator\Desktop\test.txt')

In [52]:
Data.shape

(528, 68)

In [57]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train , y_test= train_test_split(Data,label, test_size=0.3, random_state=0)

In [189]:
from sklearn.svm import SVC
clf= SVC(kernel='rbf',C=100,gamma=0.01)

In [190]:
clf.fit(x_train, y_train)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [191]:
clf.score(x_test, y_test)

0.7232704402515723

In [192]:
clf.score(x_train, y_train)

0.7506775067750677

In [227]:
from sklearn.model_selection import cross_val_score
svm= SVC(kernel='rbf',C=100,gamma=0.1,random_state=0)
cross_val_score(svm, Data, label, cv=20, scoring='accuracy').mean()

0.6839304871373837

In [228]:
from sklearn.naive_bayes import GaussianNB
NB= GaussianNB()
cross_val_score(NB, Data, label, cv=20, scoring='accuracy').mean()

0.6492346918726228

In [230]:
from sklearn. ensemble import RandomForestClassifier
RF= RandomForestClassifier(n_estimators=100)
cross_val_score(RF, Data, label, cv=20, scoring='accuracy').mean()

0.6593884678539851

In [249]:
from sklearn.linear_model import LogisticRegression
LR= LogisticRegression(random_state=0, solver='liblinear',multi_class='auto',C=100)
cross_val_score(LR, Data, label, cv=20, scoring='accuracy').mean()

0.7258304843304845

In [256]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors=7,weights='uniform')
cross_val_score(knn, Data, label, cv=20, scoring='accuracy').mean()

0.5236796169985826

In [270]:
from sklearn.ensemble import AdaBoostClassifier
Ada= AdaBoostClassifier(n_estimators=100,random_state=0, algorithm='SAMME')
cross_val_score(Ada, Data, label, cv=20, scoring='accuracy').mean()

0.5406694104107898

In [287]:
from sklearn.ensemble import GradientBoostingClassifier
GBDT= GradientBoostingClassifier(n_estimators= 100, random_state=0)
cross_val_score(GBDT, Data, label, cv=20, scoring='balanced_accuracy').mean()

0.6578231292517007

In [286]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda= LinearDiscriminantAnalysis(n_components=7)
cross_val_score(lda, Data,label, cv=20, scoring='balanced_accuracy').mean()

0.7407312925170068

In [292]:
from xgboost import XGBClassifier
xgb= XGBClassifier(n_estimators=200)
cross_val_score(xgb, Data,label, cv=20, scoring='accuracy').mean()

0.6925985642709781

In [291]:
Data.shape

(528, 68)

## 使用librosa提取MFCCs

In [320]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
wave= r'C:\Users\Administrator\Desktop\test.wav'
fs, audio=wav.read(wave)
mfcc_feature= mfcc(audio, fs)

In [322]:
mfcc_feature.T.shape

(13, 178)

In [2]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import os
import numpy as np
def MFCC(wave):
    file= os.listdir(wave)
    data= []
    label= []
    for i in file:
        each_data= []
        temp= os.path.join(wave,i)
        for j in os.listdir(temp):
            sr, y= wav.read(os.path.join(temp,j))
            feature= mfcc(y, sr).T
            each_data.append(np.concatenate((np.mean(feature,axis=1),np.std(feature, axis=1)),axis=0))
            label.append(i)
        data.append(each_data)
    return data, label
wave= r'C:\Users\Administrator\Desktop\wav'
data, label=MFCC(wave)

c:\users\administrator\anaconda3\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [6]:
f= open(r'C:\Users\Administrator\Desktop\test2.txt','w+')
for i1 in range(len(data)):
    arrs= []
    temp= data[i1]
    for i2 in range(len(temp)):
        joint= temp[i2]
        arrs.append(joint)
        for i3 in range(len(temp[0])):
            strNum= str(joint[i3])
            f.write(strNum)
            f.write(' ')
        f.write('\n')
f.close()

In [7]:
Data= np.loadtxt(r'C:\Users\Administrator\Desktop\test2.txt')

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda= LinearDiscriminantAnalysis(n_components=7)
cross_val_score(lda, Data,label, cv=20, scoring='balanced_accuracy').mean()

0.6761054421768707

In [15]:
from sklearn.preprocessing import StandardScaler
std= StandardScaler()
data= std.fit_transform(Data)

In [16]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda= LinearDiscriminantAnalysis(n_components=7)
cross_val_score(lda, data,label, cv=20, scoring='balanced_accuracy').mean()

0.6761054421768707

In [35]:
clf= LinearDiscriminantAnalysis(n_components=7)
clf.fit(x_train, y_train)
y_score= clf.predict(x_test)

In [36]:
print(classification_report(y_test, y_score, digits=5))

              precision    recall  f1-score   support

       angry    0.78049   0.88889   0.83117        36
      axiety    0.93750   0.68182   0.78947        22
      boring    0.66667   0.72727   0.69565        22
     disgust    0.63636   0.70000   0.66667        10
       happy    0.61111   0.50000   0.55000        22
     neutral    0.81481   0.88000   0.84615        25
         sad    0.90909   0.90909   0.90909        22

   micro avg    0.77358   0.77358   0.77358       159
   macro avg    0.76515   0.75530   0.75546       159
weighted avg    0.77716   0.77358   0.77054       159



In [22]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
svm= SVC(kernel='rbf',C=10,gamma=0.01,random_state=0)
cross_val_score(svm, data, label, cv=20, scoring='accuracy').mean()

0.7450929294205156

In [23]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train , y_test= train_test_split(data,label, test_size=0.3, random_state=0)

In [24]:
clf= SVC(kernel='rbf',C=10,gamma=0.01,random_state=0)
clf.fit(x_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [25]:
clf.score(x_test, y_test)

0.779874213836478

In [26]:
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score,recall_score, f1_score,roc_auc_score

In [33]:
y_score=clf.predict(x_test)

In [29]:
confusion_matrix(label,y_score)

array([[123,   0,   0,   0,   3,   0,   0],
       [  1,  62,   0,   0,   5,   0,   0],
       [  0,   1,  67,   0,   0,  10,   2],
       [  1,   2,   0,  38,   0,   2,   2],
       [ 11,   2,   0,   3,  54,   0,   0],
       [  0,   1,   4,   0,   1,  72,   0],
       [  0,   0,   2,   0,   0,   0,  59]], dtype=int64)

In [30]:
from sklearn.metrics import classification_report

In [34]:
print(classification_report(y_test, y_score, digits=5))

              precision    recall  f1-score   support

       angry    0.79545   0.97222   0.87500        36
      axiety    0.80952   0.77273   0.79070        22
      boring    0.81250   0.59091   0.68421        22
     disgust    0.75000   0.60000   0.66667        10
       happy    0.64706   0.50000   0.56410        22
     neutral    0.72414   0.84000   0.77778        25
         sad    0.87500   0.95455   0.91304        22

   micro avg    0.77987   0.77987   0.77987       159
   macro avg    0.77338   0.74720   0.75307       159
weighted avg    0.77616   0.77987   0.77079       159

